In [ ]:
from dataclasses import dataclass
@dataclass
class FakeTokenizer:
    vocab = ["management"]

from text2network.datasets.text_datasets import *

config = TrainDataConfig(
    data_path="data/preprocessed/",
    new_word_cutoff=5,
    val_items=5
)
pts = PreTrainDataSet(config)
dataset = pts.make_dataset("2000", val_items=0)
tokenizer = FakeTokenizer()


In [ ]:
pts.compute_length("2022")

In [ ]:
i

In [ ]:
dataset

In [ ]:
dd = pts._get_missing_freq_table(llm="2022", tokenizer=tokenizer)
dd

In [ ]:
dd = pts.get_missing_tokens(llm_list=[pts.llms[0]], tokenizer=tokenizer)
dd

In [ ]:
dd = pts.get_missing_tokens(llm_list=pts.llms, tokenizer=tokenizer)
dd

In [ ]:
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    llm="2022",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40)

In [ ]:
from text2network.functions.train_mlm import train_mlm_simple


train_mlm_simple(config)

In [ ]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)


from transformers import AutoModelForMaskedLM, AutoTokenizer


trainer.prep_mlm_and_tokenizer()
test_model = AutoModelForMaskedLM.from_pretrained(config.llm.model_name_or_path)
model = AutoModelForMaskedLM.from_pretrained(trainer.model_base_folder)
assert model.bert.embeddings.word_embeddings.num_embeddings > test_model.bert.embeddings.word_embeddings.num_embeddings

test_tokenizer = AutoTokenizer.from_pretrained(trainer.tokenizer_base_folder)
negtest_tokenizer = AutoTokenizer.from_pretrained(config.llm.model_name_or_path)
assert test_tokenizer.vocab != negtest_tokenizer.vocab
assert test_tokenizer.vocab["manager"]==negtest_tokenizer.vocab["manager"]

In [ ]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)

trainer.train_one_model(llm="2022")

from transformers import AutoModelForMaskedLM, AutoTokenizer

output_folder = trainer.compute_outputfolder_for_llm(config, llm="2022")
test_model = AutoModelForMaskedLM.from_pretrained(output_folder)
test_tokenizer = AutoTokenizer.from_pretrained(output_folder)
assert test_model.bert.embeddings.word_embeddings.num_embeddings == len(test_tokenizer.vocab)

In [1]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, gpu_batch=100,eval_steps=50, max_steps=500, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)


trainer.train_one_model(llm="2022")

/opt/homebrew/Caskroom/miniforge/base/envs/text2network/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/text2network/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 10%|█         | 50/500 [01:18<11:50,  1.58s/it]

{'loss': 1.3233, 'learning_rate': 4.5e-05, 'epoch': 49.0}
{'eval_loss': 2.6640539169311523, 'eval_accuracy': 0.3888888888888889, 'eval_runtime': 0.1774, 'eval_samples_per_second': 28.183, 'eval_steps_per_second': 5.637, 'epoch': 49.0}


 20%|██        | 100/500 [02:37<10:24,  1.56s/it]

{'loss': 0.3051, 'learning_rate': 4e-05, 'epoch': 99.0}
{'eval_loss': 2.2032418251037598, 'eval_accuracy': 0.7058823529411765, 'eval_runtime': 0.1186, 'eval_samples_per_second': 42.144, 'eval_steps_per_second': 8.429, 'epoch': 99.0}


 30%|███       | 150/500 [03:55<09:17,  1.59s/it]

...

 70%|███████   | 350/500 [11:10<13:49,  5.53s/it]

{'loss': 0.0528, 'learning_rate': 1.5e-05, 'epoch': 349.0}
{'eval_loss': 4.18858003616333, 'eval_accuracy': 0.4, 'eval_runtime': 0.162, 'eval_samples_per_second': 30.855, 'eval_steps_per_second': 6.171, 'epoch': 349.0}


 80%|████████  | 400/500 [19:35<21:56, 13.16s/it]  

{'loss': 0.0353, 'learning_rate': 1e-05, 'epoch': 399.0}
{'eval_loss': 3.626871109008789, 'eval_accuracy': 0.47058823529411764, 'eval_runtime': 0.1444, 'eval_samples_per_second': 34.625, 'eval_steps_per_second': 6.925, 'epoch': 399.0}


 89%|████████▉ | 446/500 [2:19:14<11:41, 13.00s/it]    